In [1]:
using Pkg
Pkg.add("CSVFiles")
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("Parquet")
Pkg.add("FileIO")
Pkg.add("DataFramesMeta")
Pkg.add("Query")
Pkg.add("TidierData")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.to

In [2]:
using CSVFiles
using CSV
using DataFrames, Query
using Parquet
using Dates
#using DataFramesMeta
using Distributed
using Base.Threads


In [2]:
df = CSV.read("/home/flex/code/code11/ml-25m/ratings.csv", DataFrame; delim=',', header=true)

Row,userId,movieId,rating,timestamp
,Int64,Int64,Float64,Int64
1,1,296,5.0,1147880044
2,1,306,3.5,1147868817
3,1,307,5.0,1147868828
4,1,665,5.0,1147878820
5,1,899,3.5,1147868510
6,1,1088,4.0,1147868495
7,1,1175,3.5,1147868826
8,1,1217,3.5,1147878326
9,1,1237,5.0,1147868839


In [ ]:
# Definir el número de partes en las que quieres dividir el archivo
N = 10


# Calcular el tamaño de cada parte
num_rows = nrow(df)
rows_per_part = div(num_rows, N)


# Crear un loop para dividir el DataFrame y guardar cada parte
for i in 0:(N-1)
    start_row = i * rows_per_part + 1
    end_row = (i == N - 1) ? num_rows : (start_row + rows_per_part - 1)


    # Crear un DataFrame para la parte actual
    part_df = df[start_row:end_row, :]


    # Guardar la parte como un nuevo archivo CSV
    CSV.write("movielens_part_$(i+1).csv", part_df)
end


println("El archivo ha sido dividido en $N partes.")


In [2]:
finales=Dict{String, Int}()
function count_ratings_by_genre(ratings, movies)
    # Unir las tablas por 'movieId'
    data = innerjoin(ratings, movies, on=:movieId)
   
    # Inicializar un diccionario para contar las calificaciones por género
    genre_count = Dict{String, Int}()
   
    for row in eachrow(data)
        genres = split(row.genres, "|")  # Dividir los géneros por '|'
        for genre in genres
            genre_count[genre] = get(genre_count, genre, 0) + 1
        end
    end
    Base.merge!(+,finales,genre_count)
    return genre_count
end

count_ratings_by_genre (generic function with 1 method)

In [3]:
movies = CSV.read("/home/flex/code/code11/ml-25m/movies.csv", DataFrame)

Row,movieId,title,genres
,Int64,String,String
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,2,Jumanji (1995),Adventure|Children|Fantasy
3,3,Grumpier Old Men (1995),Comedy|Romance
4,4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,5,Father of the Bride Part II (1995),Comedy
6,6,Heat (1995),Action|Crime|Thriller
7,7,Sabrina (1995),Comedy|Romance
8,8,Tom and Huck (1995),Adventure|Children
9,9,Sudden Death (1995),Action


In [4]:
start_time = now()
Threads.@threads for i=1:10
    ratings=CSV.read("/home/flex/code/code11/movielens_part_"*string(i)*".csv", DataFrame)
    count_ratings_by_genre(ratings, movies)
end

for (genre, count) in finales
    println("$genre: $count calificaciones")
end
end_time = now()
total_time = end_time - start_time
println("Tiempo total de ejecución: ", total_time)

War: 1267346 calificaciones
Documentary: 322449 calificaciones
Action: 7446918 calificaciones
Adventure: 5832424 calificaciones
Mystery: 2010995 calificaciones
Children: 2124258 calificaciones
Thriller: 6763272 calificaciones
Sci-Fi: 4325740 calificaciones
Comedy: 8926230 calificaciones
(no genres listed): 26627 calificaciones
IMAX: 1063279 calificaciones
Fantasy: 2831585 calificaciones
Drama: 10962833 calificaciones
Animation: 1630987 calificaciones
Western: 483731 calificaciones
Horror: 1892183 calificaciones
Romance: 4497291 calificaciones
Film-Noir: 247227 calificaciones
Crime: 4190259 calificaciones
Musical: 964252 calificaciones
Tiempo total de ejecución: 15892 milliseconds


Data analysis

In [7]:
function promedioratings(ratings, movies)
    # Unir las tablas por 'movieId'
    data1 = innerjoin(ratings, movies, on=:movieId)
   
    # Inicializar un diccionario para contar las calificaciones por género
    genre_pro= Dict{String, Float64}()
    genre_count = Dict{String, Int}()
   
    for row in eachrow(data1)
        genres = split(row.genres, "|")  # Dividir los géneros por '|'     
           
        for genre in genres
            genre_count[genre] = get(genre_count, genre, 0) + 1
            x=data1.genres.== genre
            y= data1[x,1:6].rating
            z=Base.sum(y)
            genre_pro[genre] = z
        end
    end
    return genre_pro
end

promedioratings (generic function with 1 method)

In [4]:
ratings=CSV.read("/home/flex/code/code11/movielens_part_1.csv", DataFrame)

Row,userId,movieId,rating,timestamp
,Int64,Int64,Float64,Int64
1,1,296,5.0,1147880044
2,1,306,3.5,1147868817
3,1,307,5.0,1147868828
4,1,665,5.0,1147878820
5,1,899,3.5,1147868510
6,1,1088,4.0,1147868495
7,1,1175,3.5,1147868826
8,1,1217,3.5,1147878326
9,1,1237,5.0,1147868839


In [ ]:
start_time = now()
Threads.@threads for i=1:1
    ratings=CSV.read("/home/flex/code/code11/movielens_part_"*string(i)*".csv", DataFrame)
    promedioratings(ratings, movies)
    
end

for (genre, count) in genre_count
    for (genre, pro) in genre_pro
    println("$genre: $pro/ $count promedio")
    end
end
end_time = now()
total_time = end_time - start_time
println("Tiempo total de ejecución: ", total_time)